# Part 2: Word Embeddings

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer
    
try:
    import torchtext
except:
    !pip install torchtext
    
try:
    import spacy
except:
    !pip install spacy

try:
    spacy.load('en')
except:
    !python -m spacy download en

     |████████████████████████████████| 92kB 3.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/6c/cb/69/466aef9cee879fb8f645bd602e34d45e754fb3dee2cb1a877a
Successfully built torchbearer


Word embeddings transform a one-hot encoded vector (a vector that is 0 in elements except one, which is 1) into a much smaller dimension vector of real numbers. The one-hot encoded vector is a *sparse vector*, whilst the real valued vector is a *dense vector*. 

The key concept in these word embeddings is that words that appear in similar _contexts_ appear nearby in the vector space, i.e. the Euclidean distance between these two word vectors is small. By context here, we mean the surrounding words. For example in the sentences "I purchased some items at the shop" and "I purchased some items at the store" the words 'shop' and 'store' appear in the same context and thus should be close together in vector space.

We'll talk about some of the well-known algorithms for learning embeddings in the lectures, but you might have already heard of a popular model called *word2vec*, which was first published in a rejected ICLR submission (it has some pretty damning reviews, but also has thousands of citations!). In this lab we'll use pre-trained *GloVe* vectors. *GloVe* is a different algorithm for computing word vectors, although the outcome is similar to *word2vec*. These pre-trained embeddings have been trained on a gigantic corpus. We can use these pre-trained vectors within any of our models, with the idea that as they have already learned the context of each word they will give us a better starting point for our word vectors. This usually leads to faster training time and/or improved accuracy.

In PyTorch, we use word vectors with the `nn.Embedding` layer, which takes a _**[sentence length, batch size]**_ tensor and transforms it into a _**[sentence length, batch size, embedding dimensions]**_ tensor. `nn.Embedding` layers can be trained from scratch, or they can be initialised (and optionally fixed) with pre-trained embedding data. The key thing to remember about an `nn.Embedding` is that it does not need to explicitly use a one-hot vector representation at any point; it just maps an index to a vector. This is important because it implies massive computational savings; more concretly an Emdedding is essentially a linear map in which the weight matrix of the linear layer is multiplied by a one-hot sparse-vector to produce a lower-dimensional (dense) output. This is exactly equivalent to just selecting the column of the weight matrix corresponding to the index represented by the sparse vector.

In this part of the lab we won't be training any models; instead we'll be looking at the word embeddings and investigating a few interesting things we can do with them.

## Loading the GloVe vectors

First, we'll load the pre-trained GloVe vectors. The `name` field specifies what the vectors have been trained on, here the `6B` means a corpus of 6 billion words. The `dim` argument specifies the dimensionality of the word vectors. GloVe vectors are available in 50, 100, 200 and 300 dimensions. There is also a `42B` and `840B` glove vectors, however they are only available at 300 dimensions. The first time you run this it will take time as the vectors need to be downloaded:

In [2]:
import torchtext.vocab

glove = torchtext.vocab.GloVe(name='6B', dim=100)

print(f'There are {len(glove.itos)} words in the vocabulary')

.vector_cache/glove.6B.zip: 862MB [04:59, 2.87MB/s]                           
100%|█████████▉| 398035/400000 [00:18<00:00, 21622.01it/s]

There are 400000 words in the vocabulary


As shown above, there are 400,000 unique words in the GloVe vocabulary. These are the most common words found in the corpus the vectors were trained on. **In these set of GloVe vectors, every single word is lower-case only.**

`glove.vectors` is the actual tensor containing the values of the embeddings.

In [3]:
glove.vectors.shape

torch.Size([400000, 100])

We can see what word is associated with each row by checking the `itos` (int to string) list. 

Below implies that row 0 is the vector associated with the word 'the', row 1 for ',' (comma), row 2 for '.' (period), etc.

In [4]:
glove.itos[:10]

['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s"]

We can also use the `stoi` (string to int) dictionary, in which we input a word and receive the associated integer/index. If you try get the index of a word that is not in the vocabulary, you receive an error.

In [5]:
glove.stoi['the']

0

We can get the vector of a word by first getting the integer associated with it and then indexing into the word embedding tensor with that index.

In [6]:
glove.vectors[glove.stoi['the']]

tensor([-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344,
        -0.5755,  0.0875,  0.2879, -0.0673,  0.3091, -0.2638, -0.1323, -0.2076,
         0.3340, -0.3385, -0.3174, -0.4834,  0.1464, -0.3730,  0.3458,  0.0520,
         0.4495, -0.4697,  0.0263, -0.5415, -0.1552, -0.1411, -0.0397,  0.2828,
         0.1439,  0.2346, -0.3102,  0.0862,  0.2040,  0.5262,  0.1716, -0.0824,
        -0.7179, -0.4153,  0.2033, -0.1276,  0.4137,  0.5519,  0.5791, -0.3348,
        -0.3656, -0.5486, -0.0629,  0.2658,  0.3020,  0.9977, -0.8048, -3.0243,
         0.0125, -0.3694,  2.2167,  0.7220, -0.2498,  0.9214,  0.0345,  0.4674,
         1.1079, -0.1936, -0.0746,  0.2335, -0.0521, -0.2204,  0.0572, -0.1581,
        -0.3080, -0.4162,  0.3797,  0.1501, -0.5321, -0.2055, -1.2526,  0.0716,
         0.7056,  0.4974, -0.4206,  0.2615, -1.5380, -0.3022, -0.0734, -0.2831,
         0.3710, -0.2522,  0.0162, -0.0171, -0.3898,  0.8742, -0.7257, -0.5106,
        -0.5203, -0.1459,  0.8278,  0.27

We'll be doing this a lot. __Use the following block to create a function that takes in word embeddings and a word and returns the associated vector.__ You should throw an error if the word doesn't exist in the vocabulary:

In [0]:
def get_vector(embeddings, word):
    try:  
        return glove.vectors[glove.stoi[word]]
    except:
        raise ValueError("Vector not found")

As before, we use a word to get the associated vector.

In [10]:
get_vector(glove, 'the')

tensor([-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344,
        -0.5755,  0.0875,  0.2879, -0.0673,  0.3091, -0.2638, -0.1323, -0.2076,
         0.3340, -0.3385, -0.3174, -0.4834,  0.1464, -0.3730,  0.3458,  0.0520,
         0.4495, -0.4697,  0.0263, -0.5415, -0.1552, -0.1411, -0.0397,  0.2828,
         0.1439,  0.2346, -0.3102,  0.0862,  0.2040,  0.5262,  0.1716, -0.0824,
        -0.7179, -0.4153,  0.2033, -0.1276,  0.4137,  0.5519,  0.5791, -0.3348,
        -0.3656, -0.5486, -0.0629,  0.2658,  0.3020,  0.9977, -0.8048, -3.0243,
         0.0125, -0.3694,  2.2167,  0.7220, -0.2498,  0.9214,  0.0345,  0.4674,
         1.1079, -0.1936, -0.0746,  0.2335, -0.0521, -0.2204,  0.0572, -0.1581,
        -0.3080, -0.4162,  0.3797,  0.1501, -0.5321, -0.2055, -1.2526,  0.0716,
         0.7056,  0.4974, -0.4206,  0.2615, -1.5380, -0.3022, -0.0734, -0.2831,
         0.3710, -0.2522,  0.0162, -0.0171, -0.3898,  0.8742, -0.7257, -0.5106,
        -0.5203, -0.1459,  0.8278,  0.27

## Similar Contexts

Now to start looking at the context of different words. 

If we want to find the words similar to a certain input word, we first find the vector of this input word, then we scan through our vocabulary finding any vectors similar to this input word vector.

The function below returns the closest 10 words to an input word vector:

In [0]:
import torch

def closest_words(embeddings, vector, n=10):
    distances = [(w, torch.dist(vector, get_vector(embeddings, w)).item()) for w in embeddings.itos]
    return sorted(distances, key = lambda w: w[1])[:n]

Let's try it out with 'korea'. The closest word is the word 'korea' itself (not very interesting), however all of the words are related in some way. Pyongyang is the capital of North Korea, DPRK is the official name of North Korea, etc.

Interestingly, we also get 'Japan' and 'China',  implies that Korea, Japan and China are frequently talked about together in similar contexts. This makes sense as they are geographically situated near each other. 

In [12]:
closest_words(glove, get_vector(glove, 'korea'))

[('korea', 0.0),
 ('pyongyang', 3.9039554595947266),
 ('korean', 4.068886756896973),
 ('dprk', 4.2631049156188965),
 ('seoul', 4.340494155883789),
 ('japan', 4.551243782043457),
 ('koreans', 4.615609169006348),
 ('south', 4.65822696685791),
 ('china', 4.839518070220947),
 ('north', 4.986356735229492)]

Looking at another country, India, we also get nearby countries: Thailand, Malaysia and Sri Lanka (as two separate words). Australia is relatively close to India (geographically), but Thailand and Malaysia are closer. So why is Australia closer to India in vector space? A plausible explaination is that India and Australia appear together in the context of [cricket](https://en.wikipedia.org/wiki/Cricket) matches.

In [13]:
closest_words(glove, get_vector(glove, 'india'))

[('india', 0.0),
 ('pakistan', 3.6954822540283203),
 ('indian', 4.114313125610352),
 ('delhi', 4.155975818634033),
 ('bangladesh', 4.261017799377441),
 ('lanka', 4.435845851898193),
 ('sri', 4.515716552734375),
 ('australia', 4.806082725524902),
 ('thailand', 4.994781017303467),
 ('malaysia', 5.009334087371826)]

We'll also create another function that will nicely print out the tuples returned by our closest_words function.

In [0]:
def print_tuples(tuples):
    for w, d in tuples:
        print(f'({d:02.04f}) {w}') 

Using the `print_tuples` function use the code block below to print out the 10 neighbours of 'jaguar':

In [15]:
print_tuples(closest_words(glove, get_vector(glove, 'jaguar')))

(0.0000) jaguar
(4.0384) rover
(4.2649) mustang
(4.3939) e-type
(4.4494) xk8
(4.4579) xjs
(4.4906) xj6
(4.5109) xkr
(4.5336) sepecat
(4.5409) xk120


__Use the following block to explain the results.__ (hint: use Google if you don't know what any of the terms are!)

6 of the 9 closest words refer to different car models of the Jaguar car producer. Rovers, sepecat and mustung are an exception to this. Rover is associated with the producer name, sepecat with a British-French jet attack whose complete name is "SEPECAT Jaguar" and mustung to the ford's car that presumably is compared to the jaguar one or at least they appear in the same context.

## Analogies

Another property of word embeddings is that we can apply standard arithmetic operations. This can give interesting results.

We'll show an example of this first, and then explain it:

In [0]:
def analogy(embeddings, word1, word2, word3, n=5):
    
    candidate_words = closest_words(embeddings, get_vector(embeddings, word2) - get_vector(embeddings, word1) + get_vector(embeddings, word3), n+3)
    
    candidate_words = [x for x in candidate_words if x[0] not in [word1, word2, word3]][:n]
    
    print(f'{word1} is to {word2} as {word3} is to...')
    
    return candidate_words

In [17]:
print_tuples(analogy(glove, 'man', 'king', 'woman'))

man is to king as woman is to...
(4.0811) queen
(4.6429) monarch
(4.9055) throne
(4.9216) elizabeth
(4.9811) prince


This is the canonical example which shows off this property of word embeddings. So why does it work? Why does the vector of 'woman' added to the vector of 'king' minus the vector of 'man' give us 'queen'?

If we think about it, the vector calculated from 'king' minus 'man' gives us a "royalty vector". This is the vector associated with traveling from a man to his royal counterpart, a king. If we add this "royality vector" to 'woman', this should travel to her royal equivalent, which is a queen!

We can do this with other analogies too. For example, this gets an "acting career vector":

In [18]:
print_tuples(analogy(glove, 'man', 'actor', 'woman'))

man is to actor as woman is to...
(2.8133) actress
(5.0039) comedian
(5.1399) actresses
(5.2773) starred
(5.3085) screenwriter


__Use the following block to compute a 'capital city vector' that predicts the capital of England based on the capital and name of another country__:

In [34]:
print_tuples(analogy(glove, 'netherlands', 'amsterdam', 'england'))

netherlands is to amsterdam as england is to...
(4.7527) london
(5.1758) birmingham
(5.2297) melbourne
(5.2413) manchester
(5.2575) glasgow


__Use the following block to compute an 'musical genre vector' that predicts the genre of music played by Eminem based on another musician/band and their genre__:

In [36]:
print_tuples(analogy(glove, 'stones', 'rock', 'eminem'))

stones is to rock as eminem is to...
(5.5567) rap
(5.8930) rapper
(6.0016) hop
(6.1494) gangsta
(6.3648) rappers
